# Big Data Modelling and Management 2022

Group number: 22

Students (name : student number):

1. Lucas Corrêa 20211006

2. Vera Canhoto: 20210659 

3. Doyun: 20200565

4. Bruna Duarte: 20210669

## 🚚 BDMM Second Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the second assignment is modelling. We will use the World Wide Importers database and convert it to a document-based database. To that end, we will be leveraging concepts like data denormalization, indices, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logistics). Welcome!   <br>
Even though business is thriving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements are starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data are spread accross different systems, but we've already managed to pull them all into a mongo dump file. This data file is an exact dump of the SQL data so includes all the same structure, the SQL tables become collections and the rows become documents. This means all the original SQL keys are included in the data.<br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>

Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](./WWI.png)

**Note** You can open the file WWI.png that is in the same directory as this notebook to see the above image in more detail and zoom in as you need.

Addtionally, the WWI team asked you the deliver the following outputs in **4 weeks**:
- Understand and model the database in MongoDB.
- Setup the database so that it is performs well for the queries they have provided. You should include reasoning in comments for the decisions you make on modelling the database.
- Answer the questions (queries) on the data provided.  
- Submit the results by following the instructions.  

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

Note that WWI has the following query requirements for the database.

1. The web team needs to know:  
    1. Which state province do we have the most suppliers in?  
    2. How many people have three or more `OtherLanguage`? 
    3. Top 10 most common `OtherLanguage` for people records. 
    4. How many customer records are valid after `November 2015`? 
    5. What percentage of people records don't have the UserPreferences field? 

2. The warehouse group needs to know:  
    1. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?
    2. Which items get ordered the most in bulk (largest average quantity ordered)?  
    3. Which two items get ordered together the most?
    4. For each customer category which 3 items have the ordered the most?
    5. What is the current stock of each stockgroup?

3. The CFO needs to know:  
    1. What is the monthly total order count for each month?  
    2. How many orders are there from the customer `Tailspin Toys (Head Office)`?
    3. What are the average monthly sales prices of all goods sold? 
    4. In each state province what is the average customer credit limit?   
    5. What are the yearly expenditures with each supplier (per supplier name)?  

4. Partnerships needs to know:  
    1. What is the most common payment type?  
    2. What percentage of people have their `Title` as `Team Member`?
    3. Which supplier of the category `Novelty Goods Supplier` has the most transactions?  
    4. What is the highest `CommissionRate` that a person has?

5. The marketing team needs to know:  
    1. What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?
    2. Who are the most common `PickedByPersonID` person names for orders done by customer `Adriana Pena`?
    3. How many people have in their name the string `Sara`?
    5. What are the top 10 most Common Names (Primary or Surnames) of people?

Transform the mongo dump file provided with this notebook and model a database following mongodb's best practices. You should adjust the data model to best fit the use cases provided above. Think about collections, embedding, linking, indexing, and the patterns learned in class. Provide justifications for each decision you make. What, if any, are the trade-off's or disadvantages of your approach.

Use MongoDB queries to answer the questions on your transformed database.

### Deliverables

1. Notebook with all DB creation operations and CRUD operations to create the data model. **Important** you should include in comments justification for your decisions on modelling the data.;
2. Second notebook with all required queries and answers for the questions, **Important** please indicate with comments the steps in the data model you took to optimise each query;


# Additional Information

## Groups  

Students should form groups of at least 4 and at most 5. <br>

## Submission  Deadline

The submission includes two notebooks with outputs (cells must be run). 
Please make sure to indicate:
1. group number,
2. group members with student names and numbers,
3. the name of the database that you created. <br>

Upload the notebook on moodle before **23:59 on June 22nd**

## Evaluation   

The second homework assignment counts 40% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>

Each group submission will be evaluated on three components:
1. correctness of results;
2. simplicity and performance characteristics of the solution;
3. justification of decisions.

50% -  Database design  
50% -  Query results including performance

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups with high similarity levels in their code will undergo investigation.


* [1. Setup](Setup)
* [2. Data Modeling](#DataModeling)
    * [2.1 Embedding](#Embedding)
    * [2.2 Indexing](#Indexing)

<hr>
<a id="Setup">
    
# 1. Setup
    
</a>

In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
protocol = "mongodb"
user = 'AzureDiamond'
password = 'hunter2'
host = 'localhost'
port = 27017
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}/")
db = client.WideWorldImporters
print(f"Database info: {db}\n")

Database info: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'WideWorldImporters')



In [3]:
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")
print(collection_list)

The database contains 25 collections
['orderlines', 'purchaseorders', 'stockitemsstockgroups', 'stateprovinces', 'stockitemstransactions', 'countries', 'transactiontypes', 'colors', 'invoices', 'customercategories', 'stockitems', 'orders', 'cities', 'people', 'suppliercategories', 'purchaseorderlines', 'stockgroups', 'invoicelines', 'deliverymethods', 'suppliertransactions', 'customertransactions', 'paymentmethods', 'suppliers', 'customers', 'packagetypes']


<hr>
<a id="DataModeling">
    
# 2. Data Modeling

    
</a>


<a id="Embedding">

## 2.1. Embedding
    
</a>

In [4]:
# 1.Stateprovince > City Customer
query_1 = {
        "$lookup":{
           "from": "cities",
           "localField": "PostalCityID",
           "foreignField": "CityID",
           "as": "Cities"
        }
    }
query_2 = {
    "$unwind": "$Cities"
}
query_3 = {
    "$project": {
        "CustomerID": 1,
        "StateProvinceID": "$Cities.StateProvinceID",
        "CreditLimit":1
    }
}
query_4 = { 
        "$merge" : {
            "into" : "Customers_StateProvinceID_embed",
            "whenMatched": "replace"
        }
    }
pipeline = [query_1, query_2, query_3,query_4]
r = db.customers.aggregate(pipeline)

db.Customers_StateProvinceID_embed.find_one()

{'_id': ObjectId('6287c598636e5a12693dc817'),
 'CustomerID': 1,
 'CreditLimit': None,
 'StateProvinceID': 28}

In [5]:
# 2. Stateprovince > City Supplier
query_1 = {
        "$lookup":{
           "from": "cities",
           "localField": "PostalCityID",
           "foreignField": "CityID",
           "as": "Cities"
        }
    }
query_2 = {
    "$unwind": "$Cities"
}

query_3 = {
        "$lookup":{
           "from": "stateprovinces",
           "localField": "Cities.StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "StateProvince"
        }
    }

query_4 = {
    "$unwind": "$StateProvince"
}

query_5 = {
    "$project": {
        "SupplierID": 1,
        "StateProvinceID": "$Cities.StateProvinceID",
        "CreditLimit":1,
        "StateProvinceName":"$StateProvince.StateProvinceName"
    }
}
query_6 = { 
        "$merge" : {
            "into" : "Suppliers_StateProvinceID_embed",
            "whenMatched": "replace"
        }
    }
pipeline = [query_1, query_2, query_3,query_4,query_5,query_6]
r = db.suppliers.aggregate(pipeline)
db.Suppliers_StateProvinceID_embed.find_one()

{'_id': ObjectId('6287c561636e5a12693dc7c4'),
 'SupplierID': 1,
 'StateProvinceID': 15,
 'StateProvinceName': 'Indiana'}

In [9]:
#creates a list of reference sotckIds in the order documents
query_1 = {
        "$lookup":{
           "from": "orderlines",
           "pipeline": [{'$group':{'_id':'$OrderID','items':{'$push':'$StockItemID'}}},
                        {'$project': { '_id':0,'items':1}} ],
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "OrderItemsIDs"
        }
    }

query_2 = {
    "$unwind": "$OrderItemsIDs"
}

query_3 = { '$set': 
           {'OrderItemsIDs': '$OrderItemsIDs.items'} 
          }

query_4= { 
        "$merge" : {
            "into" : "orders",
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1,query_2,query_3,query_4]
    
db.orders.aggregate(pipeline)

db.orders.find_one()

{'_id': ObjectId('6287c5ec636e5a126944ceae'),
 'OrderID': 1,
 'CustomerID': 832,
 'SalespersonPersonID': 2,
 'PickedByPersonID': None,
 'ContactPersonID': 3032,
 'BackorderOrderID': 45,
 'OrderDate': datetime.datetime(2013, 1, 1, 0, 0),
 'ExpectedDeliveryDate': datetime.datetime(2013, 1, 2, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsUndersupplyBackordered': True,
 'Comments': None,
 'DeliveryInstructions': None,
 'InternalComments': None,
 'PickingCompletedWhen': datetime.datetime(2013, 1, 1, 12, 0),
 'LastEditedBy': 7,
 'LastEditedWhen': datetime.datetime(2013, 1, 1, 12, 0),
 'OrderItemsIDs': [67]}

In [6]:
query_1 = {"$lookup": {
            'from':'suppliers', 
            "localField": 'SupplierID', 
            "foreignField": "SupplierID", 
            "as": "supplierdetails"}
          }

query_2 = {"$unwind" : "$supplierdetails"}

query_3 = {"$lookup": {
            "from":"cities",
        	"localField":"supplierdetails.DeliveryCityID",
            "foreignField":"CityID",
            "as":"cities"}
          }

query_4 = {"$unwind" : "$cities"}

query_5 = {"$lookup": {
            "from":"stateprovinces",
            "localField":"cities.StateProvinceID",
            "foreignField":"StateProvinceID",
            "as":"StateProvinces"}}

query_6 = {"$unwind" : "$StateProvinces"}

query_7 = {"$lookup": {
            "from":"suppliercategories",
            "localField":"supplierdetails.SupplierCategoryID",
            "foreignField":"SupplierCategoryID",
            "as":"Suppliercategory"}}

query_8 = {"$unwind":"$Suppliercategory"}

query_9 = {"$project": {
            "SupplierTransactionID":1,
            "SupplierID":1,
            "PaymentMethodID":1,
            "SupplierInvoiceNumber":1,
            "TransactionDate":1,
            "TransactionAmount":1,
            "supplierdetails.SupplierName":1,
            "supplierdetails.SupplierCategoryID":1,
            "cities.CityName":1,
            "cities.StateProvinceID":1,
            "StateProvinces.StateProvinceName":1,
            "Suppliercategory.SupplierCategoryName":1}}

merge = {"$merge": { 
            "into":'SuppliersTransactionsDetails', 
            "whenMatched": "replace", 
            "whenNotMatched": "insert"}
        }

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9, merge]

db.suppliertransactions.aggregate(pipeline)
db.suppliertransactions.find_one()

{'_id': ObjectId('6287c7c4636e5a12694aaa56'),
 'SupplierTransactionID': 134,
 'SupplierID': 2,
 'TransactionTypeID': 5,
 'PurchaseOrderID': 1,
 'PaymentMethodID': 4,
 'SupplierInvoiceNumber': '7290',
 'TransactionDate': datetime.datetime(2013, 1, 2, 0, 0),
 'AmountExcludingTax': Decimal128('313.50'),
 'TaxAmount': Decimal128('47.03'),
 'TransactionAmount': Decimal128('360.53'),
 'OutstandingBalance': Decimal128('0.00'),
 'FinalizationDate': datetime.datetime(2013, 1, 7, 0, 0),
 'IsFinalized': True,
 'LastEditedBy': 4,
 'LastEditedWhen': datetime.datetime(2013, 1, 7, 9, 0)}

In [35]:
query_1 = {
    '$match': {
        'CustomerName': 'Adriana Pena'
    }}

query_2 = {
        "$lookup":{
           "from": "orders",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "Orders"
        }
    }

query_3 = {
    "$unwind": "$Orders"
}


query_4 = {
    "$project": {
        '_id':0,
        'CustomerName': 1,
        'PickedByPersonID':'$Orders.PickedByPersonID'
    } }

query_5 = { 
        "$merge" : {
            "into" : "Customers_Orders_PickedByPersonID_embed",
            "whenMatched": "replace"
        } , 
    }


pipeline = [query_1,query_2,query_3,query_4,query_5]

r = db.customers.aggregate(pipeline)


<a id="Indexing">

## 2.2. Embedding
    
</a>

In [7]:
#index StockItemID
db.orderlines.create_index( [("OrderID", pymongo.DESCENDING),
                             ("StockItemID", pymongo.DESCENDING)
                            ] )

db.orders.create_index( [("OrderID", pymongo.DESCENDING)] )

db.stockitems.create_index( [("StockItemID", pymongo.DESCENDING)] )

'StockItemID_-1'

In [8]:
db.people.create_index( [("PersonID", pymongo.DESCENDING)] )

'PersonID_-1'